# 목표
**안전하면서도 실용성이 있는 비밀번호는 어떻게 하면 설정할 수 있을까?**
###### 뚫린 비밀번호들의 데이터셋을 분석해서 알아보자

#### 공격 종류
1. 무차별 대입공격
2. 사전 공격

# 해야 할 것
### 1. 양적 분석
   1) **조합 분석** => <only 숫자>, <only 영어>, <숫자+영어>, <숫자+영어+특수1개>, <숫자+영어+특수2개의 양> => 전체 비율에서 이 정도이더라. (완료)
      
      1-1) 대,소문자 포함여부 (완료)
      
   2) **길이 분석** =>  전체에서 비밀번호 길이별 뚫리는 비율. (완료)
   
   3) **길이 + 조합 분석** => 길이 8+숫자+영어+특수1개의 뚫린 비밀번호 비율 (완료)
      
### 2. 질적 분석
   1) **사용자 개인정보 포함**
   
       1-1) 이름 포함 여부 (완료)
       
       1-2) 생년월일 포함 여부
   
   2) **어떠한 특수문자가 잘 뚫리는지** (완료)
   
   3) **쿼티 키보드 가까운 문자 연속 입력 여부** (완료)
   
   4) **사전 단어 포함 여부 - 사전 공격 대비** (완료)
   
### 3. 빅데이터 전체파일로 돌려보기

### 4. 키보드 히트맵 시각화하기

In [ ]:
from dask.diagnostics import ProgressBar
import pandas as pd
import dask
import dask.array as da
import dask.dataframe as dd
import csv
import re
import math
import string

import nltk
from nltk.corpus import words
from wordcloud import WordCloud

#시각화 툴
# HoloViews에 bokeh를 백엔드 시각화 엔진으로 설정
import matplotlib.pyplot as plt
import datashader as ds
import datashader.transfer_functions as tf
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import datashade
from collections import Counter
hv.extension('bokeh')

In [ ]:
# ProgressBar 등록
progressBar = ProgressBar()
progressBar.register()
# progressBar.unregister()

In [ ]:
from dask.distributed import Client

client = Client()

print(client.dashboard_link)

num_cores = sum(client.ncores().values())

print(f"현재 사용 중인 코어 개수: {num_cores}")

# client
# client.close()

In [ ]:
# 대용량의 csv 파일을 dataframe으로 바로 불러오기.
all_data = dd.read_csv("/kaggle/input/breached-passwords/breachcompilation.txt", names=['password'], quoting=csv.QUOTE_ALL, on_bad_lines='skip', blocksize=3e8)# doublequote=True

all_data = all_data.replace('\n', '', regex=True)
all_data

# 텍스트 데이터 전처리

In [ ]:
df = all_data.head(1000000)
df

In [ ]:
df = dd.from_pandas(df, npartitions=100)  # npartitions는 필요에 따라 설정
df

In [ ]:
df = df[df['password'].apply(lambda x: isinstance(x, str), meta=('x', 'bool'))]

# 특징 추출

In [ ]:
# 비밀번호 길이 체크
df['length'] = df['password'].apply(lambda x: len(x), meta=('x', 'int'))
df

In [ ]:
# 숫자로만 이루어진 비밀번호 개수 체크하기
df['is_numeric'] = df['password'].apply(lambda x: x.isdigit(), meta=('x', 'bool'))
df

In [ ]:
# 문자로만 이루어진 비밀번호 개수 체크
df['is_alphabetic'] = df['password'].apply(lambda x: x.isalpha(), meta=('x', 'bool'))
df

In [ ]:
# 특수문자 포함 여부 체크
pattern = re.compile(r'[^a-zA-Z0-9]')
df['has_special_char_regex'] = df['password'].apply(lambda x: bool(pattern.search(x)), meta=('x', 'bool'))
df

In [ ]:
# 특수문자 개수 체크
def count_special_characters(password):
    special_characters = set(string.punctuation)
    return sum(1 for char in password if char in special_characters)

df['special_char_count'] = df['password'].apply(count_special_characters, meta=('x', 'int'))
df

In [ ]:
# 특수문자 추출
def extract_special_characters_as_string(password):
    special_chars = ''.join(set(re.findall(r'[^a-zA-Z0-9]', password)))
    return special_chars

df['all_special_chars'] = df['password'].apply(extract_special_characters_as_string, meta=('x', 'str'))
df

In [ ]:
# 반복 타입 : 반복되는 숫자 혹은 문자 타입 거르기 
# 직선 타입 : 쿼티 키보드 내 인접한 문자, 숫자 나열 거르기
def identify_pattern(password):
    repeat_pattern = re.search(r'(.)\1\1', password) is not None

    straight_pattern_alpha = re.search(r'\b(?:qwe|wer|ert|rty|tyu|yui|uio|iop|qaz|asd|sdf|dfg|fgh|ghj|hjk|jkl|zxc|xcv|cvb|vbn|bnm|m,.\s|nm,.\s|wsx|edc|ujm|zaq|xsw|cde|vfr|bgt|nhy|mju|ikl|olp|ewq|dsa|cxz)\w*\b', password.lower()) is not None
    straight_pattern_numeric = re.search(r'(012|123|234|345|456|567|678|789|890|098|987|876|765|654|543|432|321)', password) is not None

    if repeat_pattern:
        return 'repeat'
    elif straight_pattern_alpha or straight_pattern_numeric:
        return 'straight'
    else:
        return 'other'

df['pattern_type'] = df['password'].apply(identify_pattern, meta=('x', 'object'))
df

In [ ]:
# 키보드에서 입력가능한 특수문자인지 체크
def is_typeable_special_char(char):
    qwerty_special_chars = set(string.ascii_letters + string.digits + string.punctuation + ' ')
    return char in qwerty_special_chars

def filter_rows_with_typeable_special_chars(row):
    return all(is_typeable_special_char(char) for char in row['all_special_chars'])

df = df[df.apply(filter_rows_with_typeable_special_chars, axis=1, meta=('x', 'object'))]
df

In [ ]:
# 영어 대소문자 모두 들어있는지 체크
def has_both_upper_and_lower(password):
    return any(c.islower() for c in password) and any(c.isupper() for c in password)

df['has_both_upper_and_lower'] = df['password'].apply(has_both_upper_and_lower, meta=('x', 'bool'))
df

In [ ]:
# NLTK에서 영어 단어 사전 다운로드
nltk.download('words')

# 영어 단어 목록 가져오기
english_words = set(words.words())

In [ ]:
# 각 비밀번호에 영어 단어가 포함되어 있는지 여부 확인
def has_english_word(password):
    password = re.sub(r'[^a-zA-Z0-9]', ' ', password)
    return any(word.lower() in english_words for word in password.split())

df['has_english_word'] = df['password'].apply(has_english_word, meta=('x', 'bool'))

selected_columns = ['password', 'has_english_word']
selected_df = df[selected_columns]

# has_english_word True인 것만 결과 확인
has_english_word_df = selected_df[selected_df["has_english_word"]]

has_english_word_df.compute()

In [ ]:
# NLTK에서 'names' 말뭉치 - 사람 이름 데이터 다운로드
nltk.download('names')

# 'names' 말뭉치에서 남성 이름과 여성 이름 가져오기
male_names = nltk.corpus.names.words('male.txt')
female_names = nltk.corpus.names.words('female.txt')

print("Male Names:", male_names[:10])
print("Female Names:", female_names[:10])

In [ ]:
# 영어 이름 합치기
male_names = set(male_names)
female_names = set(female_names)
all_names = male_names.union(female_names)

# 각 비밀번호에 영어 이름이 포함되어 있는지 여부 확인
def has_english_name(password):
    # 특수 문자를 공백으로 대체하여 단어 추출
    password = re.sub(r'[^a-zA-Z0-9]', ' ', password)
    # 각 단어가 영어 이름 목록에 있는지 확인
    return any(word.lower() in all_names for word in password.split())

df['has_english_name'] = df['password'].apply(has_english_name, meta=('x', 'bool'))

selected_columns = ['password', 'has_english_name']
selected_df = df[selected_columns]

# has_english_name True인 것만 결과 확인
has_english_name_df = selected_df[selected_df["has_english_name"]]

has_english_name_df.compute()

In [ ]:
text_data = ' '.join(has_english_word_df['password'])

wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    prefer_horizontal = True
).generate(text_data)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# 비밀번호 강도 체크 라이브러리
!pip install zxcvbn-python

In [ ]:
# 영단어 포함 df 단어 존재시 뚫리는 강도가 실제로 약한지 라이브러리로 체크
# 2천만 개로는 메모리 부족 이슈로, 100만개로 진행
from zxcvbn import zxcvbn

def calculate_password_strength(password):
    result = zxcvbn(password)
    return result['score']

# Dask DataFrame에 비밀번호 강도 계산 추가
has_english_word_df["password_strength"] = has_english_word_df["password"].apply(calculate_password_strength, meta=('x', 'int'))

result_df = has_english_word_df[["password", "password_strength"]]
# result_df = result_df.head(1000000)
result_df

In [ ]:
very_strong_pw = result_df.loc[result_df['password_strength'] == 4].compute()
display(very_strong_pw)

strong_pw = result_df.loc[result_df['password_strength'] == 3].compute()
display(strong_pw)

middle_pw = result_df.loc[result_df['password_strength'] == 2].compute()
display(middle_pw)

weak_pw = result_df.loc[result_df['password_strength'] == 1].compute()
display(weak_pw)

very_weak_pw = result_df.loc[result_df['password_strength'] == 0].compute()
display(very_weak_pw)

In [ ]:
# 각각의 비밀번호 강도에 따른 데이터프레임의 행 개수
counts = [
    very_weak_pw.shape[0],
    weak_pw.shape[0],
    middle_pw.shape[0],
    strong_pw.shape[0],
    very_strong_pw.shape[0]
]
print(counts)

# 도넛 그래프 작성
fig, ax = plt.subplots()

ax.pie(counts, labels=['Very Weak', 'Weak', 'Middle', 'Strong', 'Very Strong'],
       autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.4), pctdistance=0.85)
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

# EDA (탐색적 데이터 분석)
시각적 및 통계적인 방법으로 데이터를 탐색

변수 간의 관계를 확인하고, 패턴이나 특이점을 발견

데이터의 분포, 중심 경향성, 편차 등을 통계적으로 분석

필요에 따라 가설 검정이나 신뢰 구간 등을 활용하여 통계적인 의사결정


In [ ]:
only_number = df["is_numeric"].sum().compute()
print('숫자로만 이루어진 데이터 개수 :', only_number)

only_alphabet = df["is_alphabetic"].sum().compute()
print('문자로만 이루어진 데이터 개수 :', only_alphabet)

has_special_char_regex = df["has_special_char_regex"].sum().compute()
print('특수문자 포함 데이터 개수 :', has_special_char_regex)

In [ ]:
# 특수문자가 포함된 비밀번호 추가 분석 진행
has_special_char_regex_df = df[["password", "length", "has_special_char_regex", "special_char_count", "all_special_chars"]].loc[df["has_special_char_regex"]]

display(has_special_char_regex_df.compute())

# 모든 문자가 숫자, 영어, 특수문자를 포함하는지 여부 계산하는 함수
def check_contains_all(value):
    has_digit = any(char.isdigit() for char in value)
    has_alpha = any(char.isalpha() for char in value)
    has_special = any(char.isalnum() == False and char.isspace() == False for char in value)
    return has_digit and has_alpha and has_special

# 모두 포함을 나타내는 열 추가
has_special_char_regex_df['contains_all'] = has_special_char_regex_df['password'].apply(check_contains_all, meta=('x', 'bool'))

contains_all_df = has_special_char_regex_df.loc[has_special_char_regex_df["contains_all"]]
contains_all = contains_all_df["contains_all"].sum().compute()

print('숫자, 영어, 특수문자 모두 포함:', contains_all)
display(contains_all_df.compute())

contains_all_length11_df = contains_all_df.loc[contains_all_df["length"] > 11].compute()
print('수,영,특 모두 포함에 길이 11 이상:', contains_all_length11_df.shape[0])
display(contains_all_length11_df)

contains_all_1 = contains_all_df.loc[contains_all_df['special_char_count'] == 1]
contains_all_1_v = contains_all_1.shape[0].compute()
print('모두 포함 중 특수문자 1개 포함:', contains_all_1_v)

display(contains_all_1.compute())

contains_all_2 = contains_all_df.loc[contains_all_df['special_char_count'] == 2]
contains_all_2_v = contains_all_2.shape[0].compute()
print('모두 포함 중 특수문자 2개 포함:', contains_all_2_v)
display(contains_all_2.compute())

contains_all_3 = contains_all_df.loc[contains_all_df['special_char_count'] > 3]
contains_all_3_v = contains_all_3.shape[0].compute()
print('모두 포함 중 특수문자 3개 이상 포함:', contains_all_3_v)
display(contains_all_3.compute())

In [ ]:
# 유출 비밀번호 구성이 어떻게 되어있는지 체크 - 숫자로만 구성된 비밀번호, 문자로만 구성된 비밀번호, 숫자+문자+특수문자
data = {
    'Label': ['only_number', 'only_alphabet', 'num+alpha+special'],
    'Value': [only_number, only_alphabet, contains_all]
}

ddf = dd.from_pandas(pd.DataFrame(data), npartitions=2)  

fig, ax = plt.subplots()

labels = ddf['Label'].compute()
values = ddf['Value'].compute()

ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.4), pctdistance=0.85)
ax.axis('equal')

plt.show()

In [ ]:
# 통계 시각화 진행
counts = [contains_all_1_v, contains_all_2_v, contains_all_3_v]

fig, ax = plt.subplots()

ax.pie(counts, labels=['special_1', 'special_2', 'special_3'],
       autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.4), pctdistance=0.85)
ax.axis('equal')

plt.show()

In [ ]:
# 패턴 타입별 개수
pattern_type = df["pattern_type"].value_counts().compute()
pattern_type

In [ ]:
has_both_upper_and_lower = df["has_both_upper_and_lower"].sum().compute()
print('대소문자 포함 데이터 개수 :', has_both_upper_and_lower)

# True인 것만, 필요한 행만 추출해서 데이터셋 보기
has_both_upper_and_lower_df = df[["password", "has_both_upper_and_lower"]].loc[df["has_both_upper_and_lower"]].compute()
has_both_upper_and_lower_df

In [ ]:
# 비밀번호 강도별 개수
password_strength_counts = result_df["password_strength"].value_counts().compute()
password_strength_counts = password_strength_counts.sort_index(ascending=True)

password_strength_counts

In [ ]:
# 비밀번호 안전도 자체 알고리즘 제작
# 영단어, 영어이름, 패턴 타입이 없는 경우에서, HOW SECURE IS YOUR PASSWORD에서 
def set_safety(row):
    if (not row['has_english_word'] and not row['has_english_name'] and row['pattern_type'] == 'other'):
        if (
            row['is_numeric'] and
            row['length'] >= 19
        ):
            return 'strong'
        elif (
            row['is_alphabetic'] and
            row['length'] >= 13
        ):
            return 'strong'
        elif (
            row['special_char_count'] >= 1 and
            row['length'] >= 11
        ):
            return 'strong'
        elif (
            row['is_numeric'] and
            row['length'] >= 16
        ):
            return 'middle'
        elif (
            row['is_alphabetic'] and
            row['length'] >= 11
        ):
            return 'middle'
        elif (
            row['special_char_count'] >= 1 and
            row['length'] >= 9
        ):
            return 'middle'
        else:
            return 'weak'
    elif (
        row['is_numeric'] and
        row['length'] >= 16
    ):
        return 'middle'
    elif (
        row['is_alphabetic'] and
        row['length'] >= 11
    ):
        return 'middle'
    elif (
        row['special_char_count'] >= 1 and
        row['length'] >= 9
    ):
        return 'middle'
    else:
        return 'weak'

df['safety'] = df.apply(set_safety, axis=1, meta=('x', 'string'))

strong_rows = df[df['safety'] == 'strong']
strong_rows.compute()

In [ ]:
# 'safety' 열의 범주별 비율 계산
safety_counts = df['safety'].value_counts().compute()
display(safety_counts)
total_samples = len(df)
display(total_samples)
safety_ratio = safety_counts / total_samples


# Matplotlib로 도넛 차트 시각화
labels = safety_ratio.index
sizes = safety_ratio.values

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.4), colors=plt.cm.Set3.colors)
ax.axis('equal')

plt.title("Safety Ratio")
plt.show()

In [ ]:
# 패턴타입 범주별 비율 계산
pattern_counts = df['pattern_type'].value_counts().compute()
total_samples = len(df)
pattern_ratio = pattern_counts / total_samples

# Matplotlib로 도넛 차트 시각화
labels = safety_ratio.index
sizes = safety_ratio.values

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.4), colors=plt.cm.Set1.colors)
ax.axis('equal')

plt.title("Pattern Ratio")
plt.show()

# 시각화

In [ ]:
length_counts = df['length'].value_counts().compute()
length_counts

In [ ]:
sorted_length_counts = length_counts.sort_index() # 정렬을 해줘야 되더라.

length_curve = hv.Curve(sorted_length_counts).opts(
    xlabel='비밀번호 길이',
    ylabel='Frequency',
    width=600,
    height=400,
    color='blue',
    tools=['hover'],
    line_width=3,  
    show_grid=True,
    title='비밀번호 길이'
)

length_curve.opts(xlim=(0, 10), ylim=(0, 100000))

display(length_curve)

#데이터 쉐이더 사용
shaded_curve = datashade(length_curve, line_width=2, cmap=['blue']).opts(
    width=600,
    height=400,  
    tools=['hover'],
    show_grid=True,
    title='비밀번호 길이',
    xlabel='길이',
    ylabel='개수'
)

shaded_curve

In [ ]:
# 비밀번호 길이 히스토그램 시각화,
length_data = df['length']

# dask array로 변환 후,
length_array = length_data.to_dask_array()

# dask array 내장 함수를 통한 히스토그램 계산
# 그저 HoloViews를 사용하니 데이터 크기 때문인지, 시각화 오류가 발생. da.histogram 사용하니 오류 없이 히스토그램 생성 완료.
histogram, edges = da.histogram(length_array, bins=20, range=(0, 20))

# HoloViews를 사용하여 히스토그램 시각화
histogram_plot = hv.Histogram((edges, histogram), kdims=['비밀번호 길이'], vdims=['개수']).opts(
        width=600, height=400,
        color='blue',
        tools=['hover'],
        show_grid=True,
        title='비밀번호 길이'
    )

histogram_plot

In [ ]:
# 특수문자 개수 시각화 진행
spec_counts = df['special_char_count'].value_counts()

spec_counts = spec_counts.compute()

# 막대 그래프로 시각화
bars = hv.Bars(spec_counts).opts(
    xlabel='특수문자',
    ylabel='개수',
    title='특수문자 수',
    show_grid=True,
    width=600,  
    height=400 
).opts(
    opts.Bars(color='orange')
)

display(bars)

# 선 그래프로 시각화
curve = hv.Curve(spec_counts).opts(
    xlabel='특수문자 수',
    ylabel='개수',
    title='특수문자 수',
    show_grid=True,
    width=600,
    height=400,
    color='blue'
)

display(curve)

# 데이터 쉐이더 사용 방식
shaded_curve = datashade(curve, line_width=2, cmap=['orange']).opts(
    width=600,
    height=400,
    tools=['hover'],
    show_grid=True,
    title='비밀번호 길이',
    xlabel='길이',
    ylabel='개수'
)

shaded_curve

In [ ]:
all_special_characters = [char for str in df['all_special_chars'] for char in str]

special_char_counts = Counter(all_special_characters)

df_counts = pd.DataFrame(list(special_char_counts.items()), columns=['Special Character', 'Count'])

df_counts = df_counts.sort_values(by='Count', ascending=False)

hv_dataset = hv.Dataset(df_counts)

bar_chart = hv.Bars(hv_dataset, 'Special Character', 'Count').opts(
    width=800,
    height=400,
    color='blue',
    xlabel='특수 문자',
    ylabel='개수',
    title='각 특수문자 수'
)

bar_chart

In [ ]:
# 여러 열에 대해 True 값만 선택한 후 바 차트 생성
selected_columns = df.select_dtypes(include='bool').columns

true_sums = {}

# 여러 열의 총합을 가지고 하나의 바 차트 생성
for column in selected_columns:
    true_sum = df[column].sum().compute()
    true_sums[column] = true_sum

bar_chart = hv.Bars(true_sums, '칼럼명', '개수')

bar_chart.opts(
    opts.Bars(width=800, height=500, tools=['hover'], xlabel='종류', ylabel='개수', title='특징')
)

In [ ]:
bars = hv.Bars(password_strength_counts).opts(
    xlabel='비밀번호 강도',
    ylabel='개수',
    title='유출 비밀번호 강도',
    show_grid=True,
    width=600,  
    height=400 
).opts(
    opts.Bars(color='blue')
)

display(bars)

In [ ]:
# Safety strong에서 분석

strong_special_count = strong_rows['special_char_count'].value_counts().compute()
# strong_special_count = strong_special_count.sort_values(by='Count', ascending=False)

bar_chart = hv.Bars(strong_special_count).opts(
    width=800,
    height=400,
    color='blue',
    xlabel='특수 문자',
    ylabel='개수',
    title='강한 safety내 각 특수문자 수'
)

bar_chart

In [ ]:
# strong 비밀번호에서만 어떤 특징이 있는지 분석

# 여러 열에 대해 True 값만 선택한 후 바 차트 생성
selected_columns = strong_rows.select_dtypes(include='bool').columns

true_sums = {}

for column in selected_columns:
    true_sum = strong_rows[column].sum().compute()
    true_sums[column] = true_sum

# 여러 열의 총합을 가지고 하나의 바 차트 생성
bar_chart = hv.Bars(true_sums, '칼럼명', '개수')

bar_chart.opts(
    opts.Bars(width=800, height=500, tools=['hover'], xlabel='종류', ylabel='개수', title='특징')
)

bar_chart

In [ ]:
# 영단어내 강도 비율 보여주기
strength_counts = password_strength_counts

total_samples = len(result_df)

strength_ratio = strength_counts / total_samples

labels = strength_ratio.index
sizes = strength_ratio.values

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.4), colors=plt.cm.Set3.colors)
ax.axis('equal')

plt.title("Strength Ratio")
plt.show()

In [ ]:
length_counts = very_strong_pw['length'].value_counts()

length_counts = length_counts.compute()
sorted_length_counts = length_counts.sort_index()

length_curve = hv.Curve(sorted_length_counts).opts(
    xlabel='비밀번호 길이',
    ylabel='Frequency',
    width=600,
    height=400,
    color='blue',
    tools=['hover'],
    line_width=3,
    show_grid=True,
    title='비밀번호 길이'
)

display(length_curve)

#데이터 쉐이더 사용 방식
shaded_curve = datashade(length_curve, line_width=2, cmap=['blue']).opts(
    width=600,    
    height=400,  
    tools=['hover'],
    show_grid=True,
    title='비밀번호 길이',
    xlabel='길이',
    ylabel='개수'
)

shaded_curve

## 통계분석

In [ ]:
length_stats = df['length'].describe().compute()
display(length_stats)

result = df.groupby('safety')['length'].agg(['mean', 'std', 'min', 'max']).compute().sort_index()
display(result)

print("안전과 특수문자 관계 비교")
result = df.groupby('safety')['special_char_count'].agg(['mean', 'std', 'min', 'max']).compute().sort_index()
display(result)

print("영어 포함")
safety_english = df.groupby('safety')['has_english_word'].value_counts().compute().sort_index()
display(result)

safety_both = df.groupby('safety')['has_both_upper_and_lower'].value_counts().compute().sort_index()
display(result)

safety_pattern_type = df.groupby('safety')['pattern_type'].value_counts().compute().sort_index()
display(result)


In [ ]:
# 전체 데이터 내 안전 비밀번호의 비율 계산
filtered_df = df.loc[
    (df['length'] >= 11) &
    (df['special_char_count'] >= 2) &
    (~df['all_special_chars'].isin(['_', '.', '!', '@'])) &
    (df['pattern_type'] == 'other') &
    (df['has_english_word'] != False)
]

# 필터링된 행의 개수
filtered_count = filtered_df.shape[0].compute()

# 전체 행의 개수
total_count = df.shape[0].compute()

# 비율 계산
ratio = filtered_count / total_count

labels = ['Filtered', 'Remaining']
sizes = [ratio, 1 - ratio]
colors = ['#ff9999', '#66b3ff']
explode = (0.1, 0)

# 비율 시각화
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors, explode=explode)
plt.axis('equal')
plt.title('Filtered Rows Ratio')
plt.show()


In [ ]:
import numpy as np

# 'length' 열의 발생 빈도 계산
length_counts = df['length'].value_counts()

# 엔트로피 계산
total_passwords = df['length'].count().compute()
entropy = -(length_counts / total_passwords * np.log2(length_counts / total_passwords)).sum().compute()

print("Password Length Entropy:", entropy)

In [ ]:
# GPU 사용 위해 DASK-CUDA 사용 시도, 윈도우는 사용 불가, 캐글에서만 적용되나, 오류 존재.
!pip install dask-cuda

In [ ]:
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import cudf

# CUDA 클러스터 설정
cluster = LocalCUDACluster(n_workers=1)
client = Client(cluster)

import cudf

# GPU 데이터프레임 생성
gdf = cudf.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

# GPU 데이터프레임 출력
gdf